<a href="https://colab.research.google.com/github/lima-breno/deep_learning_frameworks/blob/main/pytorch_vae_com_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Autoencoder Variacional**

Um autoencoder variacional (VAE) é um tipo de rede neural que combina elementos de autoencoders e modelos generativos probabilísticos. Ao contrário de autoencoders tradicionais, um VAE mapeia os exemplos de entrada para uma distribuição de probabilidade no espaço latente em vez de mapeá-los para pontos específicos nesse espaço. Isso significa que, em vez de apenas aprender uma representação comprimida dos dados de entrada, o VAE aprende a distribuição das representações latentes.

O VAE consiste em duas partes principais: um codificador, que mapeia as entradas para distribuições no espaço latente, e um decodificador, que reconstrói as entradas a partir das amostras aleatórias tiradas dessa distribuição. Durante o treinamento, o VAE tenta minimizar a diferença entre a distribuição das representações latentes dos exemplos de treinamento e uma distribuição de referência, geralmente uma distribuição normal multivariada.

Uma vez treinado, o VAE pode ser usado para gerar novas amostras, amostrando aleatoriamente do espaço latente e passando essas amostras pelo decodificador. Isso permite que o VAE aprenda uma distribuição rica e contínua dos dados de entrada, possibilitando a geração de novas amostras que se assemelham aos dados de treinamento.



In [ ]:
# Para rodar esse código, precisamos ir em EDITAR/Configurações de notebook/ SELECIONAR TPU
# DEPois disto, instalar o Plytorch XLA
!pip install -q torch_xla[tpu] -f https://storage.googleapis.com/pytorch-tpu-release/wheels/tpuvm/torch_xla.html

In [ ]:
import torch_xla
import torch_xla.core.xla_model as xm

In [ ]:
#criando o device, para usar o TPU
device = xm.xla_device()
device

device(type='xla', index=0)

###**Import das bibliotecas**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

###**Criação do modelo**

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(in_features=784, out_features=400)
        self.fc_mu = nn.Linear(400, 20)  # Camada de média do espaço latente
        self.fc_logvar = nn.Linear(400, 20)  # Camada de log-variância do espaço latente
        self.fc2 = nn.Linear(20, 400)
        self.fc3 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        mu = self.fc_mu(h1)
        logvar = self.fc_logvar(h1)
        return mu, logvar #retorna vetor de médias e de variancias

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std #é uma operação elemento-wise
        return z

    def decode(self, z): #reconstrução do valor desconstruido no encode
        h2 = F.relu(self.fc2(z)) # Assign the result to h2
        return F.sigmoid(self.fc3(h2))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1,28*28))
        z = self.reparameterize(mu,logvar)
        return self.decode(z), mu, logvar # Return mu and logvar as well

###**Criação da função de perda (Kullback-Leibler Divergence)**

In [ ]:
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x,
                                 x.view(-1, 784), #valor esperado
                                 reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

###**Criação do dataset e dataloader**

In [ ]:
transform = transforms.ToTensor()
train_data = datasets.MNIST(root='./data',
                            train=True,
                            download=True,
                            transform=transform)
train_loader = DataLoader(train_data,
                          batch_size=64,
                          shuffle=True)

###**Configuração dos Hiperparâmetros**

In [ ]:
learning_rate = 1e-3
epochs = 60

model = VAE().to(device) #PARA USAR O TPU
optimizer = optim.Adam(
    params = model.parameters(),
    lr = learning_rate
)

###**Execução do treino**

In [ ]:
import tqdm

In [ ]:
%%time
model.train()
loss_history = []
for epoch in range(epochs):
    total_loss = 0
    for batch_idx, (data, _) in tqdm.tqdm(enumerate(train_loader)):
        data = data.to(device)
        recon_batch,mu,logvar = model(data)
        loss = loss_function(
            recon_x = recon_batch,
            x = data,
            mu = mu,
            logvar = logvar
            )
        loss.backward()
        xm.optimizer_step(optimizer)
        optimizer.zero_grad()
        total_loss += loss.item()


        if batch_idx % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, epochs, batch_idx+1, len(train_loader), loss.item() / len(data)))

loss_history.append(total_loss / len(train_data))
print('Epoch [{}/{}], Average Loss: {:.4f}'.format(epoch+1, epochs, total_loss / len(train_data)))


1it [00:00,  1.31it/s]

Epoch [1/60], Step [1/938], Loss: 548.3981


67it [14:46, 13.23s/it]


KeyboardInterrupt: 

###**Função para gerar novos dados**

In [ ]:
def generate_digit():
  with torch.no_grad():
      z = torch.randn(1, 20)
      reconstructed_img = model.decode(z)
      plt.imshow(reconstructed_img.view(28, 28).numpy(),
                 cmap='gray')
      plt.axis('off')
      plt.show()